# Hyperparameter Fine-Tuning

In [1]:
!pip install transformers==4.17 # this will also install tokenizers 
!pip install datasets
!pip install ray==1.10.0

     |████████████████████████████████| 3.8 MB 7.1 MB/s 
     |████████████████████████████████| 596 kB 41.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.2 MB/s 
     |████████████████████████████████| 136 kB 43.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.9 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 52.9 MB/s 
     |████████████████████████████████| 271 kB 62.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, roc_curve, auc
from nltk.tokenize import sent_tokenize
import pandas as pd
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaForSequenceClassification,RobertaModel, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
import datasets
from ray.tune.schedulers import PopulationBasedTraining,ASHAScheduler
from ray import tune
from ray.tune import uniform, CLIReporter
from random import randint
from sklearn.metrics import accuracy_score
# from ray.tune.stopper import TrialPlateauStopper
import os
import ray
print(ray.__version__)
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1.10.0
There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


Load Data

In [3]:
data = datasets.load_dataset("SetFit/sst2")

Using custom data configuration SetFit--sst2-c66a8eb4897c1f8f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/SetFit--sst2-c66a8eb4897c1f8f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
### number of labels
num_labels = len(np.unique(data['train']['label']))
print (num_labels)

2


Hyperparameters for model

In [6]:
# Hyper-parameters
model_name = "roberta-base" 
max_length = 50
batch_size = 32
n_epochs = 10
lr = 3e-5
warmup = 0.06
beta_1 = 0.9
beta_2 = 0.98
adam_ep = 1e-6
weight_decay = 0.1
optimizer = AdamW


Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir = "."
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Dataset class

In [13]:
class SST2Dataset():
## Dataset in pandas dataframe format##
    def __init__(self, tokenizer, df, max_length):
        super(SST2Dataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq)
        input_ids += padding   
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len)
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids

    def get_input(self,df):
        tweet = df['text'].values
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]


Function to load data for individual hyper-parameter tuning

In [8]:
def load_data(max_length,batch_size):
    if load_data  == None:
        datasets.load_dataset("SetFit/sst2")
    data_train = datasets.Dataset.to_pandas(data["train"])
    data_val = datasets.Dataset.to_pandas(data["validation"])
    data_test = datasets.Dataset.to_pandas(data["test"])
    ## Emotion dataset
    train_dataset = SST2Dataset(tokenizer, data_train, max_length)
    val_dataset = SST2Dataset(tokenizer, data_val, max_length)
    test_dataset = SST2Dataset(tokenizer, data_test, max_length)
    ## dataloader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
    return train_loader,val_loader,test_loader,len(data["train"]["text"])

Train and Eval function

In [9]:
def model_init(model_name,optimizer,data_len):
  model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = 2)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
          {
                  'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                  'weight_decay':weight_decay
          },
          {
                  'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                  'weight_decay':0.0
          }
  ]
  optimizer = optimizer(optimizer_grouped_parameters, lr=lr, betas = (beta_1,beta_2),eps = adam_ep)
  total_steps = int(n_epochs * (data_len/ batch_size))
  warmup_steps = int(warmup * total_steps)
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)
  
  return model,optimizer,scheduler


In [10]:
def set_seed(seed_value = 0):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
def train(train_loader, model, optimizer, scheduler):
    model.train()
    train_losses = []
    correct_preds = 0
    tqdm_loader = tqdm(train_loader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (tqdm_loader):
        model.zero_grad()
        

        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segments = segments.to(device)
        labels = labels.to(device)

        loss,logits = model(input_ids, attention_mask,segments,labels = labels)[:2] # forward pass (batch_size, 1)
        probs = nn.functional.softmax(logits,dim = -1)
        _,out_classes = probs.max(dim =1)
        correct = (out_classes == labels).sum()
        correct_preds += correct.item()
        train_losses.append(loss.item())

        loss.backward() # backward pass
        ## clip grad
        optimizer.step() # weights update
        if scheduler is not None:
          scheduler.step()
        optimizer.zero_grad()
        
    train_loss = np.mean(train_losses)
    epoch_accuracy = correct_preds /len(train_loader.dataset)

    return train_loss,epoch_accuracy

In [11]:
def eval(loader, model):
    model.eval()
    val_losses = []
    val_probs = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()

    val_labels = []
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)

        with torch.no_grad():
            valid_loss,valid_logits = model(input_ids, attention_mask,segments,labels = labels)[:2]
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = valid_probs.max(dim =1)
            correct = (out_classes == labels).sum()
            acc_correct += correct.item()
            val_probs.extend(valid_probs[:,1].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    valid_accuracy = acc_correct/(len(loader.dataset))

    
    fpr, tpr, threshold = roc_curve(val_labels, val_probs)
    val_auc = 100 * auc(fpr, tpr)

    return val_loss, val_auc, val_probs, val_labels,valid_accuracy

**Fine-Tune individual Hyperparameters**

Batch size and Learning rate

Using larger batch sizes allow higher learning rate to be used, similarly, smaller batch size with lower learning rate

In [ ]:
batch_size_set = [8,16,64]
lr_set = [1e-5,2e-5,5e-5]

In [ ]:
### Seed all test to seed 0
seed_val = 0
set_seed(seed_value = seed_val)
for i in range(len(batch_size_set)):
  batch_size = batch_size_set[i]
  lr = lr_set[i]
  train_loader,val_loader,test_loader,data_len = load_data(max_length,batch_size)
  model,optimizer,scheduler = model_init(model_name,optimizer,data_len)
  model.to(device)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc= eval(val_loader, model)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}".format(val_loss, val_auc,val_acc*100))
  print ('--'*89)
  print ('Fine-Tuning of batch size : {}, lr : {}'.format(batch_size,lr))
  print ('--'*89)
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler)

      # Validation round
      val_loss, val_auc, _, _,val_acc= eval(val_loader, model)
      val_aucs.append(val_auc)
      val_accs.append(val_acc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,np.max(np.array(val_accs)),np.argmax(np.array(val_accs*100))))
      # scheduler.step(val_acc)


      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")
  _, test_auc, test_probs, test_labels,test_acc = eval(test_loader, model) # val_loss, val_auc, val_probs, val_labels,val_acc
  print("Test AUC of batch size {}, lr {}: {:.2f}".format(batch_size,lr,test_auc))
  print ('Test ACC of batch size {}, lr {}: {:.2f}'.format(batch_size,lr,test_acc*100))



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi


Epoch 0 (before training) val loss: 0.6931, AUC: 55.61, ACC: 50.9174
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Fine-Tuning of batch size : 8, lr : 1e-05
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

New epoch, epoch 1 / 10


  0%|          | 0/865 [00:00<?, ?it/s]

Training: loss: 0.4113, accuracy 77.8468
Validation: loss: 0.2406, AUC: 97.16, ACC : 91.9725, (best ACC: 0.9197 at epoch 1)
lr : 9.574468085106385e-06
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/865 [00:00<?, ?it/s]

...

Training: loss: 0.0131, accuracy 99.5665
Validation: loss: 0.3870, AUC: 97.50, ACC : 92.5459, (best ACC: 0.9289 at epoch 7)
lr : 1.0275319567354965e-05
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/109 [00:00<?, ?it/s]

Training: loss: 0.0098, accuracy 99.7399
Validation: loss: 0.3869, AUC: 97.57, ACC : 93.0046, (best ACC: 0.9300 at epoch 9)
lr : 4.9164208456243854e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/109 [00:00<?, ?it/s]

Training: loss: 0.0019, accuracy 99.9566
Validation: loss: 0.4137, AUC: 97.58, ACC : 93.0046, (best ACC: 0.9300 at epoch 9)
lr : 0.0
!! New best val ACC !!
Test AUC of batch size 64, lr 5e-05: 98.74
Test ACC of batch size 64, lr 5e-05: 94.45


Weight Decay

In [ ]:
## larger and smaller weight decay
weight_decay_set = [0.01,0.05,0.15,0.2]

In [ ]:
# Default hyperparameters
model_name = "roberta-base" 
max_length = 50
batch_size = 32
n_epochs = 10
lr = 3e-5
warmup = 0.06
beta_1 = 0.9
beta_2 = 0.98
adam_ep = 1e-6
weight_decay = 0.1

### Seed all test to seed 0
seed_val = 0
set_seed(seed_value = seed_val)
for i in range(len(weight_decay_set)):
  weight_decay = weight_decay_set[i]
  train_loader,val_loader,test_loader,data_len = load_data(max_length,batch_size)
  model,optimizer,scheduler = model_init(model_name,optimizer,data_len)
  model.to(device)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc= eval(val_loader, model)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}".format(val_loss, val_auc,val_acc*100))
  print ('--'*89)
  print ('Fine-Tuning of weight decay : {}'.format(weight_decay))
  print ('--'*89)
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler)

      # Validation round
      val_loss, val_auc, _, _,val_acc= eval(val_loader, model)
      val_aucs.append(val_auc)
      val_accs.append(val_acc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,np.max(np.array(val_accs)),np.argmax(np.array(val_accs*100))))
      # scheduler.step(val_acc)


      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")
  _, test_auc, test_probs, test_labels,test_acc = eval(test_loader, model) # val_loss, val_auc, val_probs, val_labels,val_acc
  print("Test AUC of weight decay {}: {:.2f}".format(weight_decay,test_auc))
  print ('Test ACC of of weight decay {}: {:.2f}'.format(weight_decay,test_acc*100))



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi


Epoch 0 (before training) val loss: 0.6931, AUC: 55.61, ACC: 50.9174
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Fine-Tuning of weight decay : 0.01
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

New epoch, epoch 1 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.4229, accuracy 78.2948
Validation: loss: 0.2186, AUC: 97.09, ACC : 92.0872, (best ACC: 0.9209 at epoch 1)
lr : 2.8701426463354646e-05
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

...

Training: loss: 0.0128, accuracy 99.5954
Validation: loss: 0.4629, AUC: 97.26, ACC : 92.3165, (best ACC: 0.9289 at epoch 5)
lr : 6.286276438760452e-06
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0073, accuracy 99.8266
Validation: loss: 0.4543, AUC: 97.55, ACC : 93.0046, (best ACC: 0.9300 at epoch 9)
lr : 3.08411214953271e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0040, accuracy 99.9133
Validation: loss: 0.4849, AUC: 97.54, ACC : 93.1193, (best ACC: 0.9312 at epoch 10)
lr : 0.0
!! New best val ACC !!
Test AUC of weight decay 0.2: 98.81
Test ACC of of weight decay 0.2: 94.34


SGD optimizer

In [ ]:
lr_set = [1e-3,2e-3,3e-3,1e-2]

In [ ]:
# Default hyperparameters
model_name = "roberta-base" 
max_length = 50
batch_size = 32
n_epochs = 10
warmup = 0.06
momentum = 0.9 # seems to work well with this value
weight_decay = 0.1

### Test using SGD optimizer
### Seed all test to seed 0
seed_val = 0
set_seed(seed_value = seed_val)
for i in range(len(lr_set)):
  lr = lr_set[i]
  train_loader,val_loader,test_loader,data_len = load_data(max_length,batch_size)
  # model, optimizer and scheduler
  model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = 2)
  optimizer = optim.SGD(model.parameters(),lr = lr,momentum = momentum,weight_decay = weight_decay)
  total_steps = int(n_epochs * (data_len/ batch_size))
  warmup_steps = int(warmup * total_steps)
  model.to(device)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc= eval(val_loader, model)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}".format(val_loss, val_auc,val_acc*100))
  print ('--'*89)
  print ('Fine-Tuning of lr : {}'.format(lr))
  print ('--'*89)
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler)

      # Validation round
      val_loss, val_auc, _, _,val_acc= eval(val_loader, model)
      val_aucs.append(val_auc)
      val_accs.append(val_acc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,np.max(np.array(val_accs)),np.argmax(np.array(val_accs*100))))
      # scheduler.step(val_acc)


      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")
  _, test_auc, test_probs, test_labels,test_acc = eval(test_loader, model) # val_loss, val_auc, val_probs, val_labels,val_acc
  print("Test AUC of lr {}: {:.2f}".format(lr,test_auc))
  print ('Test ACC of of lr {}: {:.2f}'.format(lr,test_acc*100))



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie


Epoch 0 (before training) val loss: 0.6931, AUC: 55.61, ACC: 50.9174
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Fine-Tuning of lr : 0.001
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

New epoch, epoch 1 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.6085, accuracy 64.0462
Validation: loss: 0.3760, AUC: 93.37, ACC : 83.8303, (best ACC: 0.8383 at epoch 1)
lr : 0.0079390063944909
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

...

Training: loss: 0.6921, accuracy 51.9653
Validation: loss: 0.6931, AUC: 50.00, ACC : 50.9174, (best ACC: 0.5092 at epoch 0)
lr : 0.0
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.6929, accuracy 51.4740
Validation: loss: 0.6930, AUC: 50.00, ACC : 50.9174, (best ACC: 0.5092 at epoch 0)
lr : 0.0
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.6925, accuracy 52.1676
Validation: loss: 0.6930, AUC: 50.00, ACC : 50.9174, (best ACC: 0.5092 at epoch 0)
lr : 0.0
!! New best val AUC !!
!! New best val ACC !!
Test AUC of lr 0.01: 50.00
Test ACC of of lr 0.01: 49.92


Load data function for Automatic fine-tuning

In [8]:
def load_data(config,data_dir = None):
    if load_data  == None:
        datasets.load_dataset("SetFit/sst2")
    data_train = datasets.Dataset.to_pandas(data["train"])
    data_val = datasets.Dataset.to_pandas(data["validation"])
    data_test = datasets.Dataset.to_pandas(data["test"])
    ## Emotion dataset
    train_dataset = SST2Dataset(tokenizer, data_train, config['max_length'])
    val_dataset = SST2Dataset(tokenizer, data_val, config['max_length'])
    test_dataset = SST2Dataset(tokenizer, data_test, config['max_length'])
    ## dataloader
    # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
    # val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    # test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
    print (len(data["train"]["text"]))
    return train_dataset,val_dataset,test_dataset,len(data["train"]["text"])

Train function

In [9]:
def train_function(config, checkpoint_dir='fine_tuned_model', data_dir=None):

    # if checkpoint_dir == None:
    #   checkpoint_dir = 'fine-tuned_model'
    ## define model
    seed_value = 0
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = 2)
    # model = FinetuneModel(model,d_in,config['d_h'],d_out,config['dropout'],freeze_pretrained) #pretrained_model, d_in, d_h, d_out, freeze_pretrained=False

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
    model.to(device)
    print (f'is GPU available? {torch.cuda.is_available()}')
    print (f'device used: {device}')
### Optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
            {
                    'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                    'weight_decay':config['weight_decay']
            },
            {
                    'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                    'weight_decay':0.0
            }
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=config['lr'],betas = (config['beta1'],config['beta2']),eps = config['eps'])
    criterion = nn.CrossEntropyLoss()

    # if checkpoint_dir:
    #     model_state, optimizer_state = torch.load(
    #         os.path.join(checkpoint_dir, "checkpoint"))
    #     model.load_state_dict(model_state)
    #     optimizer.load_state_dict(optimizer_state)

    trainset, valset,_,len_data = load_data(config,data_dir)
    ## scheduler
    total_steps = int(config['num_epochs'] * (len_data / config['batch_size'])) # get total_steps for scheduler
    warmup_steps = int(config['warmup'] * total_steps)
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)

    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=int(config["batch_size"]),
        shuffle=True)
    valloader = torch.utils.data.DataLoader(
        valset,
        batch_size=int(config["batch_size"]),
        shuffle=False)
    ## Training ####
    for epoch in range(config['num_epochs']):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        total = 0
        correct = 0
        for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments,batch_labels) in enumerate (trainloader):
            # get the inputs; data is a list of [inputs, labels]
            model.zero_grad()

            input_ids = batch_seqs
            attention_mask = batch_seq_masks
            labels = batch_labels

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            loss,outputs = model(input_ids,attention_mask,labels = labels)[:2]
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # loss = criterion(outputs, labels)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm = config['max_gradient_norm'])
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if batch_index % 10 == 0:  # print every 2000 mini-batches
                print("Epoch: {}, batch no: {} loss: {:.2f}, accuracy: {:.2f}%".format(epoch + 1, batch_index + 1,
                                                running_loss / epoch_steps,(correct/total)*100))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        acc_correct = 0
        correct = 0
        running_loss = 0
        print ('------'*89)
        print ('Start Validating')
        print ('------'*89)
        for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments,batch_labels) in enumerate (valloader):
            # get the inputs; data is a list of [inputs, labels]
            with torch.no_grad():
                input_ids = batch_seqs
                attention_mask = batch_seq_masks
                labels = batch_labels

                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                loss,outputs = model(input_ids,attention_mask,labels = labels)[:2]
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                acc_correct += (predicted == labels).sum().item()
                correct = acc_correct
                # loss = criterion(outputs, labels)

                val_loss += loss.cpu().numpy()
                val_steps += 1

            ### printing item
            running_loss += loss.item()
            if batch_index % 10 == 0:  # print every 2000 mini-batches
                print("Epoch: {}, batch no: {} loss: {:.2f}, accuracy: {:.2f}%".format(epoch + 1, batch_index + 1,
                                                running_loss / val_steps,(correct/total)*100))
                running_loss = 0.0
                
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            print (f'checkpoint path : {path}')
            torch.save((model.state_dict(), optimizer.state_dict(),scheduler.state_dict()), path)
            print ('Model Saved!')
        accuracy = correct/(len(valloader.dataset))
        tune.report(loss=(val_loss / val_steps), accuracy=accuracy)
        ## scheduler
    # test_acc,test_loss = test_accuracy(model, device) # evaluate on test set
    # tune.report(loss = test_loss,accuracy = test_acc)
    print("Finished Training")

Test accuracy function for test set

In [10]:
def test_accuracy(config,model, device="cpu"):
    _,_,testset,_ = load_data(config)

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=config['batch_size'], shuffle=False)
    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    test_loss = 0.
    test_steps = 0
    with torch.no_grad():
        for batch_index, (batch_seqs, batch_seq_masks,batch_seq_segments, batch_labels) in enumerate (testloader):
      
            batch_seqs, batch_seq_masks,batch_labels = batch_seqs.to(device), batch_seq_masks.to(device), batch_labels.to(device)
            loss,outputs = model(batch_seqs,batch_seq_masks,labels = batch_labels)[:2]
            _, predicted = torch.max(outputs.data, 1)
            # loss = criterion(outputs,batch_labels)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
            test_loss += loss.cpu().numpy()
            test_steps += 1

    return correct / total, test_loss/test_steps

In [14]:
config = {
    "num_epochs": tune.randint(3,10),
    "lr": tune.loguniform(1e-5, 1e-4),
    "batch_size": tune.choice([8,16,32,64]),
    "weight_decay": tune.uniform(0.0, 0.15),
    "beta1" : tune.uniform(0.8,0.95),
    "beta2" : tune.uniform(0.995,0.999),
    "eps": tune.loguniform(1e-8,1e-6),
    "max_gradient_norm": tune.randint(7,12),
    "dropout": tune.choice([0.1,0.2,0.3]),
    "warmup": tune.choice([0.04,0.06,0.08,0.1,0.12]),
    "max_length": tune.choice([50,100])
}

In [15]:
def main(num_samples=10,max_num_epochs =10, gpus_per_trial=1):
    # load_data()
    # ray.init()
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    # stopper = TrialPlateauStopper(metric = 'accuracy',metric_threshold=0.95, mode = 'max')
    #PBT scheduler
    # scheduler = PopulationBasedTraining(
    #     metric="accuracy",
    #     mode="max",
    #     perturbation_interval=2,
    #     hyperparam_mutations={
    #       "num_epochs": tune.randint(3,10),
    #       "weight_decay": tune.uniform(0.0, 0.1),
    #       "lr": tune.loguniform(1e-5, 1e-4),
    #       "batch_size": tune.choice([ 8, 16,32]),
    #       "beta1" : tune.uniform(0.85,0.95),
    #       "beta2" : tune.uniform(0.995,0.999),
    #       "eps": tune.loguniform(1e-8,1e-6),
    #       "max_gradient_norm": tune.randint(7,12)
    # })
    ## ASHA scheduler
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    result = tune.run(
        train_function,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter = reporter,
        )

    best_trial = result.get_best_trial("accuracy", "max", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state,scheduler_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc,test_loss = test_accuracy(best_trial.config,best_trained_model, device)
    print("Best trial test set accuracy: {} , test loss : {}".format(test_acc,test_loss))
    print (f'scheduler : {scheduler_state.keys()}')


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=20,max_num_epochs=10, gpus_per_trial=1)

Output hidden; open in https://colab.research.google.com to view.